In [ ]:
from bs4 import BeautifulSoup

html_content = """
<div class="parent">
    <div class="child">Child 1</div>
    <div class="child">Child 2</div>
    <div class="child">Child 3</div>
</div>
"""

soup = BeautifulSoup(html_content, 'html.parser')

# 特定の親divを取得
parent_div = soup.find('div', class_='parent')

# 親divの中のすべての子divをリストとして取得
child_divs = parent_div.find_all('div', class_='child')

# 子divのテキストをリストとして取得
child_texts = [child.text for child in child_divs]

print(child_texts)

In [2]:
import requests
from bs4 import BeautifulSoup
from retry import retry
import urllib
import time
import logging

import numpy as np

In [3]:
# SUUMOを東京都23区のみ指定して検索して出力した画面のurl(ページ数フォーマットが必要)
url = "https://suumo.jp/jj/bukken/ichiran/JJ010FJ001/?ar=030&bs=011&ta=13&jspIdFlg=patternShikugun&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13106&sc=13107&sc=13108&sc=13118&sc=13121&sc=13122&sc=13123&sc=13109&sc=13110&sc=13111&sc=13112&sc=13114&sc=13115&sc=13120&sc=13116&sc=13117&sc=13119&kb=1&kt=9999999&mb=0&mt=9999999&ekTjCd=&ekTjNm=&tj=0&cnb=0&cn=9999999&srch_navi={{2}}&page={}"


# リクエストがうまく行かないパターンを回避するためのやり直し
@retry(tries=3, delay=10, backoff=2)
def load_page(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    return soup

In [98]:
import re

# 面積を抽出する関数
def extract_area(text):
    match = re.search(r'(\d+(\.\d+)?)m2', text)
    if match:
        return match.group(1)
    return None

extract_area("50.00m2　（15.13m2）")

def extract_name(text):
    return text.split()[0]

extract_name("給田西住宅\u3000１号棟")

def extract_price(text):
    match = re.search(r'(\d+億)?\d+万円', text)
    if match:
        return match.group(0)
    return None

In [51]:
soup = load_page(url.format(2))

In [ ]:
def get_apidata_suumo():
    # SUUMOを東京都23区のみ指定して検索して出力した画面のurl(ページ数フォーマットが必要)
    url = "https://suumo.jp/jj/bukken/ichiran/JJ010FJ001/?ar=030&bs=011&ta=13&jspIdFlg=patternShikugun&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13106&sc=13107&sc=13108&sc=13118&sc=13121&sc=13122&sc=13123&sc=13109&sc=13110&sc=13111&sc=13112&sc=13114&sc=13115&sc=13120&sc=13116&sc=13117&sc=13119&kb=1&kt=9999999&mb=0&mt=9999999&ekTjCd=&ekTjNm=&tj=0&cnb=0&cn=9999999&srch_navi={{2}}&page={}"
    ESTATES_MAX=30
    info={"name":[],
          "price":[],
          "address":[],
          "area":[],
          "age":[]}

    for page in range(1,2):
        soup = load_page(url.format(page))
        estates_groups = soup.find("div",class_='property_unit_group')
        estates = estates_groups.find_all('div', class_='property_unit')

        for i in range(ESTATES_MAX):
            estate=estates[i].find_all('div', class_='dottable-line')

            
            name_text=estate[0].find_all("dd")[0].text
            info["name"].append(extract_name(name_text))
            price_text=estate[1].find_all("dd")[0].text
            info["price"].append(extract_price(price_text))
            info["address"]=estate[2].find_all("dd")[0].text
            area_text=estate[3].find_all("dd")[0].text
            info["area"]=extract_area(area_text)
            info["age"]=estate[4].find_all("dd")[1].text
            


        print(info)


    return 

get_apidata_suumo()

{'name': ['ロクタス', 'コーポ東山', '洗足シュロス', 'グリーンプラザ綾瀬II', '豊栄アルベルゴ上野', 'パークハイツ', '池袋シティハウス', '鷺宮プリンスマンション', '新菊川ビル', '久我山ロータリーマンション', 'イトーピア豊玉マンション', 'ハイネス阿佐谷', 'ダイヤモンドスクエア駒形', 'セザール第二竹ノ塚', 'セザール第二竹ノ塚', 'ワコー徳丸マンション', '給田西住宅', 'グリーンパーク花畑Iパサージュ', 'アイディ上野Ⅲ', 'ビレッタ池之端', 'ライオンズマンション青戸', 'スカイタウン中野', 'シャンボール上北沢', '力建杉並ハイム', 'カーサ常盤台', 'ボナハイツ中川C棟', 'グリーンコーポお花茶屋', 'ワコーレ赤塚公園', '久我山ロータリーマンション119号室', 'ダイアパレス三ノ輪'], 'price': ['1億4700万円', '1100万円', '1400万円', '1400万円', '1480万円', '1480万円', '1490万円', '1500万円', '1540万円', '1580万円', '1580万円', '1590万円', '1650万円', '1690万円', '1690万円', '1880万円', '1890万円', '1980万円', '1980万円', '1980万円', '1980万円', '1980万円', '1990万円', '1990万円', '2080万円', '2090万円', '2090万円', '2150万円', '2180万円', '2190万円'], 'address': '東京都荒川区荒川１-４６－７', 'area': '41', 'age': '1979年9月'}


In [58]:
estates_groups = soup.find("div",class_='property_unit_group')
estates = estates_groups.find_all('div', class_='property_unit')

In [ ]:
for j in range(6):
    a=estates[1].find_all('div', class_='dottable-line')[j].find_all("dd")

    a_text=[item.text for item in a]

    print(a_text)
    print("--------------------------")

In [79]:
a='給田西住宅\u3000１号棟'

a.split("\u")


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 0-1: truncated \uXXXX escape (1715940244.py, line 3)

In [ ]:
child_divs = div_estates.find_all('div', class_='property_unit')
len(child_divs)

In [ ]:
child_texts = [child.text for child in child_divs]

In [ ]:
child_divs[0]